In [1]:
# Models
#from LoadData.models import Experiment, Sample, Dna, Vector, Measurement, Inducer, LoadProcess
# Data handling
import pandas as pd
import numpy as np
import json
import os
import openpyxl as opxl
from itertools import islice
import csv
import subprocess

In [2]:
def fix_synergy_time(df):
    t = np.array([])
    for i, value in enumerate(df['Time']):
        if i > 0:
            if df['Time'].iloc[i].hour < df['Time'].iloc[i-1].hour:
                t = np.append(t, [24 + value.hour + value.minute/60 + value.second/3600])
            else:
                t = np.append(t, [value.hour + value.minute/60 + value.second/3600])
        else:
            t = np.append(t, [value.hour + value.minute/60 + value.second/3600])
    df['Time'] = t

In [24]:
# Cargo Metadata desde el archivo JSON
data_str = open('MetaTest.json').read()
data = json.loads(data_str)
df_json = pd.read_json(data)
columns = [x+str(y) for x in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for y in range(1,13)]
df_json.columns = columns
df_json.index = ['Strain', 'Media', 'DNA']

#If el usuario no me da el nombre del experimento, le pongo el nombre del archivo de datos
experiment_name = os.path.basename('ExpTest.xlsx').split('/')[-1].split('.')[0]

In [244]:
# Estas medidas deben venir dadas por el usuario y yo debo sumar "Results"
medidas = ['OD600:600', 'RFP-YFP:585/10,620/15', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20', 'Results']

wb = opxl.load_workbook(filename = 'ExpTest.xlsx', data_only=True)
ws = wb['Data']
machine_name = ws['B'][8].value + str(ws['B'][9].value)

# Completar esta lista para todos los posibles valores de longitudes de onda
name_map = {'OD600:600':'OD', 'RFP-YFP:500/27,540/25':'YFP', 'CFP:420/50,485/20':'CFP',
'RFP-YFP:585/10,620/15':'RFP'}

In [245]:
def find_index(col, measures):
    return [(celda.value, celda.row, opxl.utils.column_index_from_string(celda.column))
        for celda in col
        if celda.value in measures]

In [246]:
lista_rows = find_index(ws['A'], medidas)
print(lista_rows)
ws.delete_rows(0, lista_rows[0][1] - 1)
lista_rows2 = find_index(ws['A'], medidas)
print(lista_rows2)
ws.delete_rows(lista_rows2[len(lista_rows2)-1][1]-1, 150)
lista_rows3 = find_index(ws['A'], medidas)
print(lista_rows3)

[('OD600:600', 47, 1), ('RFP-YFP:585/10,620/15', 148, 1), ('RFP-YFP:500/27,540/25', 249, 1), ('CFP:420/50,485/20', 350, 1), ('Results', 451, 1)]
[('OD600:600', 1, 1), ('RFP-YFP:585/10,620/15', 102, 1), ('RFP-YFP:500/27,540/25', 203, 1), ('CFP:420/50,485/20', 304, 1), ('Results', 405, 1)]
[('OD600:600', 1, 1), ('RFP-YFP:585/10,620/15', 102, 1), ('RFP-YFP:500/27,540/25', 203, 1), ('CFP:420/50,485/20', 304, 1)]


### Data extraída como array

In [247]:
raw_data = np.array(list(ws.values))
raw_data = np.delete(raw_data, 2, 1)
raw_data

array([['OD600:600', None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, 'Time', 'A1', ..., 'H10', 'H11', 'H12'],
       ...,
       [None, datetime.time(23, 44, 4), 8085, ..., 84208, 84569, 49625],
       [None, datetime.time(23, 59, 4), 8049, ..., 84107, 84952, 49304],
       [None, datetime.datetime(1900, 1, 1, 0, 14, 4), 8049, ..., 84596,
        85809, 49175]], dtype=object)

In [155]:
lista_rows3

[('OD600:600', 1, 1),
 ('RFP-YFP:585/10,620/15', 102, 1),
 ('RFP-YFP:500/27,540/25', 203, 1),
 ('CFP:420/50,485/20', 304, 1)]

In [198]:
# Lo hago solo con el primero porque para el resto serán iguales
dim = lista_rows[1][1] - lista_rows[0][1] - 4
dim

97

In [204]:
meas_name = np.array(dim)

In [205]:
meas_name

array(97)

In [236]:
strArr = np.empty(dim, dtype='object')
strArr[:] = name_map[lista_rows[0][0]]

In [237]:
name_map[lista_rows[0][0]]

'OD'

array([['OD600:600', None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, 'Time', 'A1', ..., 'H10', 'H11', 'H12'],
       ...,
       [None, datetime.time(23, 44, 4), 8085, ..., 84208, 84569, 49625],
       [None, datetime.time(23, 59, 4), 8049, ..., 84107, 84952, 49304],
       [None, datetime.datetime(1900, 1, 1, 0, 14, 4), 8049, ..., 84596,
        85809, 49175]], dtype=object)

array([['OD600:600', None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, 'Time', 'A1', ..., 'H10', 'H11', 'H12'],
       ...,
       [None, datetime.time(23, 44, 4), 8085, ..., 84208, 84569, 49625],
       [None, datetime.time(23, 59, 4), 8049, ..., 84107, 84952, 49304],
       [None, datetime.datetime(1900, 1, 1, 0, 14, 4), 8049, ..., 84596,
        85809, 49175]], dtype=object)

In [242]:
raw_data

array([['OD600:600', None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, 'Time', 'T° OD600:600', ..., 'H10', 'H11', 'H12'],
       ...,
       [None, datetime.time(23, 44, 4), 37, ..., 84208, 84569, 49625],
       [None, datetime.time(23, 59, 4), 37, ..., 84107, 84952, 49304],
       [None, datetime.datetime(1900, 1, 1, 0, 14, 4), 37, ..., 84596,
        85809, 49175]], dtype=object)

In [239]:
strArr.T

array(['OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD',
       'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD', 'OD'], dtype=object)

In [182]:
np.delete(raw_data, 2, 1)

array([['OD600:600', None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, 'Time', 'A1', ..., 'H10', 'H11', 'H12'],
       ...,
       [None, datetime.time(23, 44, 4), 8085, ..., 84208, 84569, 49625],
       [None, datetime.time(23, 59, 4), 8049, ..., 84107, 84952, 49304],
       [None, datetime.datetime(1900, 1, 1, 0, 14, 4), 8049, ..., 84596,
        85809, 49175]], dtype=object)

In [183]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}, index=['a', 'b', 'c'])
df

,A,B
a,1,4
b,2,5
c,3,6


In [184]:
df.to_numpy()

AttributeError: 'DataFrame' object has no attribute 'to_numpy'

In [188]:
a = np.array(df.values)
a

array([[1, 4],
       [2, 5],
       [3, 6]])

In [190]:
b = np.array([[7, 8], [9, 10]])
b

array([[ 7,  8],
       [ 9, 10]])

In [194]:
a = np.append(a, b, 0)
a

array([[ 1,  4],
       [ 2,  5],
       [ 3,  6],
       [ 7,  8],
       [ 9, 10]])

### Data extraída como un DataFrame

In [146]:
ws.delete_rows(0, 2)

In [147]:
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)

In [148]:
df = pd.DataFrame(data, columns=cols)
df = df.drop('T° OD600:600', axis=1)

In [150]:
# Falta saber a qué medida (CFP, OD, etc) corresponde cada dato

In [128]:
df_OD = pd.DataFrame(df.iloc[0:lista_rows2[0][1] - 3])
df_OD['name'] = 'OD'
fix_synergy_time(df_OD)

KeyError: 'Time'

In [10]:
df_RFP = pd.DataFrame(df.iloc[lista_rows2[0][1] + 1:lista_rows2[1][1] - 3])
df_RFP['name'] = 'RFP'
fix_synergy_time(df_RFP)
df_RFP.index = range(97)
#df_RFP

In [11]:
df_YFP = pd.DataFrame(df.iloc[lista_rows2[1][1] + 1:lista_rows2[2][1] - 3])
df_YFP['name'] = 'YFP'
fix_synergy_time(df_YFP)
df_YFP.index = range(97)
#df_YFP

In [12]:
df_CFP = pd.DataFrame(df.iloc[lista_rows2[2][1] + 1:lista_rows2[3][1] - 3])
df_CFP['name'] = 'CFP'
fix_synergy_time(df_CFP)
df_CFP.index = range(97)
#df_CFP

In [14]:
dfs = [df_OD, df_RFP, df_YFP, df_CFP]

# 1) Experiment
e = Experiment(name=experiment_name, machine=machine_name)
e.save()

# 2) Sample
# get experiment_id
# row, col, media, strain

# 3) DNA
# name

# Empiezo a recorrer los platillos que están en df_json: indexes: strain, media, dna:
for col_name, col_serie in df_json.iteritems():
    # experiment_id
    existing_dna = [i.name for i in Dna.objects.all()]

    plate_row = col_name[0]
    plate_col = col_name[1:]
    st = col_serie['Strain']['value']
    med = col_serie['Media']['value']

    s = Sample(experiment=e, row=plate_row, col=plate_col, media=med, strain=st)
    s.save()

    DNA_name = col_serie['DNA']['value']
    if DNA_name != 'None':
        if DNA_name not in existing_dna:
            d = Dna(name=DNA_name, sboluri='')
            d.save()
            v = Vector(dna=d, sample=s)
            v.save()
        else:
            d = Dna.objects.filter(name__exact=DNA_name)[0]
            v = Vector(dna=d, sample=s)
            v.save()
    # 4) Measurement
    # name, value, time
    for df in dfs:
        for i, value in enumerate(df[col_name]):
            nam = df['name'].iloc[i]
            val = value
            #t = df['Time'].iloc[i]
            t = i
            m = Measurement(sample=s, name=nam, value=val, time=t)
            m.save()


NameError: name 'Experiment' is not defined